In [1]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [2]:
from mukkeBude.model import MukkeBudeTransformer
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
import numpy as np
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
# Create mappings
mapping = MusicMapping.create()
mapping.save("mapping.txt")

In [4]:
# Load songs
songs = utils.read_all("./dataset/Pokemon/")
# paths = paths[:100]
print(f"Found {len(songs)} songs in corpus.")

encoded_songs = []
for song in songs:
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.append(mapping.textify(encoded_song))

print(f"Songs encoded: {len(encoded_songs)}")

Found 9 songs in corpus.


d:\Projekte\Python\Tensorflow\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:137: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
d:\Projekte\Python\Tensorflow\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:138: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,


Songs encoded: 9


In [5]:
# Create dataset
utils.create_train_data(encoded_songs, "raw_pokemon_train_ds.txt")
print("Dataset created")

Dataset created


In [6]:
# Train model
model = MukkeBudeTransformer(mapping)
print(model)
model.train("raw_pokemon_train_ds.txt", min_training_seq_len=32, epochs=50)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        780288    
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 transformer_decoder_2 (Tran  (None, None, 256)        394749

In [17]:
# Create song
generated_song = model.generate("xxbos", max_length=1024, probability=0.25)
new_song_str = generated_song.numpy().decode("utf-8")
print(f"CLEAN SONG: {new_song_str}\n\n")

# THIS DEPENDS ON THE OUTPUT - NEED TO FIX THE OUTPUT
# new_song_str = new_song_str[15:]
# new_song_str = "xxbos xxpad " + new_song_str
# new_song_str = new_song_str.replace("   ", " ")

print(new_song_str)

prompt_tokens: [2]
CLEAN SONG: xxbos n69 d1 n53 d1 xxsep d1 n57 d1 xxsep d1 n71 d1 n60 d1 xxsep d1 n64 d1 xxsep d1 n72 d1 n69 d1 xxsep d2 n76 d4 n71 d4 n65 d4 xxsep d4 n72 d2 n69 d2 xxsep d2 n69 d2 n67 d2 xxsep d2 n67 d2 n62 d2 xxsep d2 n67 d1 n52 d1 xxsep d1 n55 d1 xxsep d1 n69 d1 n59 d1 xxsep d1 n62 d1 xxsep d1 n71 d1 n67 d1 xxsep d2 n74 d4 n64 d4 n57 d4 xxsep d4 n72 d2 n69 d2 xxsep d2 n71 d2 n64 d2 xxsep d2 n72 d2 n57 d2 xxsep d2 n69 d1 n50 d1 xxsep d1 n53 d1 xxsep d1 n71 d1 n57 d1 xxsep d1 n60 d1 xxsep d1 n72 d1 n69 d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 n69 d1 xxsep d1 xxsep d1 n64 d4 xxsep d1 n71 d1 xxsep d1 xxsep d1 n77 d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 n77 d1 n63 d1 n64 d4 n72 d1 xxsep d1 xxsep d1 n69 d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d2 n60 d1 xxsep d1 n69 d1 n71 d1 n69 d1 n69 d1 xxsep d1 n64 d1 n69 d1 n75 d1 xxsep d1 n69 d1 n69 d1 xxsep d1 xxsep d1 xxsep d1 n69 d

In [19]:
# Convert to music21
new_song_ints = mapping.numericalize(new_song_str.split(" "))
new_song_ints = np.array(new_song_ints)

print(new_song_ints)
print(mapping.textify(new_song_ints))

new_song = utils.from_polyphonic_encoding(new_song_ints, mapping, bpm=120, validate=True)
path = Path("generated_song_pokemon.mid")
utils.write_midi(new_song, path)
utils.write_musicxml(new_song, "generated_song_pokemon.musicxml")

[  0  73 133 ...   3 133  64]
xxbos n69 d1 n53 d1 xxsep d1 n57 d1 xxsep d1 n71 d1 n60 d1 xxsep d1 n64 d1 xxsep d1 n72 d1 n69 d1 xxsep d2 n76 d4 n71 d4 n65 d4 xxsep d4 n72 d2 n69 d2 xxsep d2 n69 d2 n67 d2 xxsep d2 n67 d2 n62 d2 xxsep d2 n67 d1 n52 d1 xxsep d1 n55 d1 xxsep d1 n69 d1 n59 d1 xxsep d1 n62 d1 xxsep d1 n71 d1 n67 d1 xxsep d2 n74 d4 n64 d4 n57 d4 xxsep d4 n72 d2 n69 d2 xxsep d2 n71 d2 n64 d2 xxsep d2 n72 d2 n57 d2 xxsep d2 n69 d1 n50 d1 xxsep d1 n53 d1 xxsep d1 n71 d1 n57 d1 xxsep d1 n60 d1 xxsep d1 n72 d1 n69 d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 n69 d1 xxsep d1 xxsep d1 n64 d4 xxsep d1 n71 d1 xxsep d1 xxsep d1 n77 d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 n77 d1 n63 d1 n64 d4 n72 d1 xxsep d1 xxsep d1 n69 d1 xxsep d1 xxsep d1 xxsep d1 xxsep d1 xxsep d2 n60 d1 xxsep d1 n69 d1 n71 d1 n69 d1 n69 d1 xxsep d1 n64 d1 n69 d1 n75 d1 xxsep d1 n69 d1 n69 d1 xxsep d1 xxsep d1 xxsep d1 n69 d1

In [ ]:
# Open song in MuseScore (Linux if installed)
!musescore4portable {path.absolute()}